# Worksheet 07

Name: Pranesh Jayasundar
UID: U08334002

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [18]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.438456696722695, 2.553321522924431, 4.016619713769143, 4.041020628919053, 4.5593390749345675, 4.486122433214509, 4.931651890673766, 4.150482313219718, 3.4900602232108158, 3.8560492501059924]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [19]:
mean = 8
stdev = 1

c2 = np.random.normal(mean, stdev, 10).tolist()
print(c2)

[8.865349700929933, 5.971769858626196, 8.388169011323757, 8.113271594758002, 7.538138087390978, 8.25324512558597, 7.994358349953638, 7.867662099241416, 6.2354358720797425, 7.000343071179394]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [20]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.000343071179394, 6.2354358720797425, 3.8560492501059924, 7.867662099241416, 7.994358349953638, 3.4900602232108158, 8.25324512558597, 4.150482313219718, 7.538138087390978, 8.113271594758002]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- The mean for c1 is 5 and stdev for c1 is 1. 
- The mean for c2 is 8 and stdev for c2 is 1.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [21]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.000343071179394, 6.2354358720797425, 7.867662099241416, 7.994358349953638, 8.25324512558597, 7.538138087390978, 8.113271594758002]
[3.8560492501059924, 3.4900602232108158, 4.150482313219718]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 7.571779171455591,  mean_2 = 3.8321972621788425
var_1 = 0.44817344474436854,  var_2 = 0.0729773481593262


/Users/praneshjayasundar/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [22]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] *  prob_c[0] / prob_x)
    prob_c1_x.append( pdf_i[1] *  prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.000343071179394
probability of observing that point if it came from cluster 0 =  0.39486212719419544
probability of observing that point if it came from cluster 1 =  0.0
point =  6.2354358720797425
probability of observing that point if it came from cluster 0 =  0.01044330400576396
probability of observing that point if it came from cluster 1 =  1.7701354577766063e-235
point =  3.8560492501059924
probability of observing that point if it came from cluster 0 =  1.054936206012334e-15
probability of observing that point if it came from cluster 1 =  5.18233195404682
point =  7.867662099241416
probability of observing that point if it came from cluster 0 =  0.7158434034001386
probability of observing that point if it came from cluster 1 =  0.0
point =  7.994358349953638
probability of observing that point if it came from cluster 0 =  0.5707030397864765
probability of observing that point if it came from cluster 1 =  0.0
point =  3.4900602232108158
probability of observing that po

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [23]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7000000001137219,  P(C_2) = 0.29999999988627807
mean_1 = 7.571779170899766,  mean_2 = 3.832197262058193
var_1 = 0.44817344657321156,  var_2 = 0.07297734814858772


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [24]:
# essentially just a copy paste of part e
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

7.000343071179394
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 0.0

6.2354358720797425
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 7.264265136687942e-234

3.8560492501059924
Probability of coming from C_1 = 4.749827961439436e-16
Probability of coming from C_2 = 0.9999999999999996

7.867662099241416
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 0.0

7.994358349953638
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 0.0

3.4900602232108158
Probability of coming from C_1 = 2.19352143221549e-14
Probability of coming from C_2 = 0.9999999999999781

8.25324512558597
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 0.0

4.150482313219718
Probability of coming from C_1 = 1.1371972847924089e-09
Probability of coming from C_2 = 0.9999999988628027

7.538138087390978
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 0.0

8.113271594758002
Probability of coming fro

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [25]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[0, 0, 1, 0, 0, 1, 0, 1, 0, 0]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.